In [ ]:
from api_util import read_token, get_headers
import requests
from datetime import datetime, timezone

# BASE_URL = 'http://localhost:3000'
token = read_token()

def get_date_time_str():
    now = datetime.now(timezone.utc)
    return now.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3] + 'Z'


def _send_request(method, endpoint, params=None, json_data=None, quiet=False):
    """
    統一的 HTTP 請求處理核心
    :param method: "GET", "POST", "PUT", "DELETE" 等
    :param endpoint: API 路徑
    :param params: GET 請求的 Query Parameters
    :param json_data: POST/PUT 請求的 Body 資料
    """
    # 確保 URL 拼接邏輯一致 (處理可能多餘或缺少的斜線)
    url = f"{BASE_URL.rstrip('/')}/{endpoint.lstrip('/')}"
    
    try:
        # 使用 requests.request 動態決定方法
        response = requests.request(
            method=method,
            url=url,
            headers=get_headers(token),
            params=params,
            json=json_data
        )
        
        # 建議一併檢查狀態碼
        response.raise_for_status()
        result = response.json()
        
        if not quiet:
            print(f"[{method}] {endpoint} Success: {result}")
            
        return result

    except requests.exceptions.RequestException as err:
        print(f"Request failed ({method} {endpoint}): {err}")
        return None

def get_all_news(data):
    res = _send_request(endpoint='/news', params=data, method="GET")
    return res.get('data')

def post_news(data):
    res = _send_request(endpoint='/news', json_data=data, method='POST')
    return res.get('data')

def bulk_create_news(data):
    res = _send_request(endpoint='/news/bulkCreate', json_data=data, method="POST")
    return res.get('data')

def delete_news(id):
    try:
        response = requests.delete(f"{BASE_URL}/news/{id}", headers=get_headers(token))
        result = response.json()
        return result
    except requests.exceptions.RequestException as err:
        print(f"Request failed: {err}")
        return None

def update_news(id, data):
    try:
        response = requests.put(f"{BASE_URL}/news/{id}", headers=get_headers(token), json=data)
        result = response.json()
        return result
    except requests.exceptions.RequestException as err:
        print(f"Request failed: {err}")
        return None

In [ ]:
# Meta 在路易斯安那州興建的 $270 億美元 超級資料中心（Hyperion）規模驚人；同時，科技公司對電力與水的需求引發了環境組織的強烈關注。
# 微軟、亞馬遜、Google、Meta 五大巨頭預計到 2026 年在 AI 基礎設施上的支出將突破 1 兆美元。投資者開始擔心，如果應用端無法快速變現，這場軍備競賽將拖累企業的利潤。
# {
#   "content": "市場傳聞 Meta 正測試代號為「芒果」（Mango）與「酪梨」（Avocado）的新一代多模態大模型，預計於明年上半年正式推出。",
#   "contentEn": "Market rumors suggest Meta is testing next-gen multimodal models codenamed 'Mango' and 'Avocado,' with an official launch expected in the first half of next year.",
#   "status": "published",
#   "publishedAt": "2025-12-24T11:28:00.000Z",
# }
post_news();

In [ ]:
update_news('12', {
    'contentEn': 'According to estimates from Wolfe Research, the Total Addressable Market (TAM) for WhatsApp\'s business messaging opportunity could reach $30 billion to $40 billion, a market that remains currently underappreciated',
    'status': 'published',
  })

In [ ]:
# draft published
get_all_news({"page": 1, "size": 100, "status": "published"})

In [ ]:
bulk_create_news([
    {
        "content": "Musk 估計，特斯拉每年將需要 1000 億到 2000 億顆 AI 晶片，這遠超當前供應鏈的極限。",
        "contentEn": "Musk estimates that Tesla will require 100 to 200 billion AI chips annually, a figure that far exceeds the limits of the current supply chain.",
        "status": "published",
        "publishedAt": get_date_time_str()
    }
])

In [ ]:
delete_news(6195)